使用MindQuantum模拟下列线路。

In [1]:
# 构造线路
import numpy as np                                          # 导入numpy库并简写为np
from mindquantum.core.gates import X, Y, Z, H, RX, RY, RZ   # 导入量子门H, X, Y, Z, RX, RY, RZ
from mindquantum.core.circuit import Circuit     # 导入Circuit模块，用于搭建量子线路

CIRCUIT = Circuit()                              # 初始化量子线路
CIRCUIT += H.on(0)                               # H门作用在第0位量子比特
print(CIRCUIT)

# 模拟线路
from mindquantum.simulator import Simulator    # 从mindquantum.simulator中导入Simulator类
sim = Simulator('mqvector', 1)   #声明一个一比特的mqvector模拟器

sim.apply_circuit(CIRCUIT)  #作用一个量子线路，当线路是一个参数化量子线路时，我们还需要提供参数值。
print(sim)

q0: ──H──
mqvector simulator with 1 qubit (little endian).
Current quantum state:
√2/2¦0⟩
√2/2¦1⟩


/home/zxh/.conda/envs/tensor_network/lib/python3.8/site-packages/mindquantum/__init__.py:25: UserWarning: MindSpore not installed, you may not be able to use hybrid quantum classical neural network.
  from . import algorithm, config, core, engine, framework, io, simulator, utils


接下来，我们用tensorq模拟同样的线路

首先在`circuit_MQ.py`文件中构建量子线路，然后用tensorq中的QuantumCircuit将量子子线路转换成张量网络

In [2]:
from tensorq import QuantumCircuit
qc_n1 = QuantumCircuit(n = 1,twoqubit_simplify = False,fname = 'circuit_MQ', circuit_package='MindQuantum') # 

其中，n = 1表示1个比特；twoqubit_simplify = False表示不进行两比特门优化，因为线路中没有两比特门；fname = 'circuit'表示线路在文件`circuit.py`中。

用tensorq中的search_order搜索缩并方案

In [3]:
from tensorq import search_order
contract_scheme = search_order(n = 1,device = 'cuda',qc = qc_n1, sc_target = 16)

Finish find contraction tree, next construct the scheme.
Finish construct the scheme.
time complexity (tc,log10), space complexity (sc,log2), memory complexity (mc) =  (0.7781512503836436, 2.0, 1.1139433523068367)


其中，n = 1表示1个比特；device = 'cuda' 表示用GPU进行计算，用CPU计算使用device = 'cpu'；qc = qc_n1 输入之前定义的量子线路。

sc_target = 16表示限制最大张量小于16GB内存，这是tensorq中很核心的一个参数，可以根据自己的设备内存进行设置。同样的缩并任务，内存越大时间复杂度越低，内存越小将会对张量做更多的分解导致更多的浮点运算，这是用时间换空间的办法解决量子模拟中的指数墙困难。

search_order会打印该缩并方案的复杂度，时间复杂度为浮点数运算次数，以10为底的对数表示；空间复杂度为最大张量的元素个数（与sc_target单位并不相同），以2为底的对数表示；空间复杂度为最大张量元素个数乘以单个数据的存储大小（与数据类型有关）。

最后，用tensorq的contraction_single_task对张量进行缩并。

In [4]:
from tensorq import contraction_single_task
tensors, scheme, slicing_indices, bitstrings = contract_scheme
result = contraction_single_task(
    tensors,
    scheme,
    slicing_indices
)
print('fianl amplitue:', result)

fianl amplitue: tensor([0.7071+0.j, 0.7071+0.j])


contract_scheme中包含了缩并张量tensors，缩并步骤scheme，切片指标slicing_indices（与张量分解有关），计算振幅的比特串bitstrings，将这些变量输入到contraction_single_task，就会返回张量网络的缩并结果，就是全部末态比特串的振幅。
